In [1]:
# Unidad: Clasificación - Sesión 1
## Ejercicio 1: Preparar el ambiente de trabajo
# Imporatamos la triada clásica 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# seaborn
import seaborn as sns
#statsmodels para modelación
import statsmodels.api as sm
import statsmodels.formula.api as smf
#Importe LogisticRegression, train_test_split, 
#StandardScaler, classification_report y roc_curve de sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
#Gráficos estilo seaborn
plt.style.use('seaborn')


In [2]:
#-----------------------------------#
#Ejercicio 2
#Leer el csv
df=pd.read_csv('southafricanheart.csv')

In [3]:
df=df.drop("Unnamed: 0", axis=1)

In [4]:
df.head(5)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [5]:
## 1.- Recodifique famhist a dummy, asignando 1 a la categoría minoritaria.
print(df['famhist'].value_counts('%'))

Absent     0.584416
Present    0.415584
Name: famhist, dtype: float64


In [6]:
df['famhist']=df['famhist'].replace(['Absent', 'Present'], [0,1])

In [7]:
print(df['famhist'].value_counts('%'))

0    0.584416
1    0.415584
Name: famhist, dtype: float64


In [8]:
# 2.- Utilice smf.logit para estimar el modelo.

In [9]:
m1_logit = smf.logit('chd ~ famhist', df).fit()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


In [10]:
def concise_summary(mod, print_fit=True):
    #guardamos los parámetros asociados a estadísticas de ajuste
    fit = pd.DataFrame({'Statistics': mod.summary2().tables[0][2][2:],
                       'Value': mod.summary2().tables[0][3][2:]})
    # guardamos los parámetros estimados por cada regresor.
    estimates = pd.DataFrame(mod.summary2().tables[1].loc[:, 'Coef.': 'Std.Err.'])
    # imprimir fit es opcional
    if print_fit is True:
        print("\nGoodness of Fit statistics\n", fit)
    
    print("\nPoint Estimates\n\n", estimates)

In [11]:
concise_summary(m1_logit)


Goodness of Fit statistics
         Statistics       Value
2             BIC:    574.1655
3  Log-Likelihood:     -280.95
4         LL-Null:     -298.05
5     LLR p-value:  4.9371e-09
6           Scale:      1.0000
7                             

Point Estimates

               Coef.  Std.Err.
Intercept -1.168993  0.143106
famhist    1.168993  0.203255


In [12]:
def inverse_logit(valor_x):
    estimate_y = m1_logit.params['Intercept']+(m1_logit.params['famhist'] * valor_x)
    return 1/(1+np.exp(-estimate_y))


In [13]:
print("La probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria es:", round(inverse_logit(1), 2))

La probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria es: 0.5


In [14]:
print("La probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria es:", round(inverse_logit(0), 2))

La probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria es: 0.24


In [15]:
#¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?
round(round(inverse_logit(1), 2) - round(inverse_logit(0), 2), 3)

0.26

In [16]:
#La probabilidad de tener una enfermedad coronaria baja un 26% cuando no se tiene antecedentes familiares.

In [17]:
#Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados. 

In [18]:
m1_ols = smf.ols('chd ~ famhist', df).fit()

In [19]:
concise_summary(m1_ols)


Goodness of Fit statistics
             Statistics     Value
2                 BIC:  601.4437
3      Log-Likelihood:   -294.59
4         F-statistic:     36.86
5  Prob (F-statistic):  2.66e-09
6               Scale:   0.21050

Point Estimates

               Coef.  Std.Err.
Intercept  0.237037  0.027922
famhist    0.262963  0.043313


In [20]:
print("\n OLS")
concise_summary(m1_ols, print_fit= False)
print("\n LOG")
concise_summary(m1_logit, print_fit=False)


 OLS

Point Estimates

               Coef.  Std.Err.
Intercept  0.237037  0.027922
famhist    0.262963  0.043313

 LOG

Point Estimates

               Coef.  Std.Err.
Intercept -1.168993  0.143106
famhist    1.168993  0.203255


In [21]:
1.16/4

0.29

In [22]:
#Es una aproximación del coeficiente estimado de la reg lineal (OLS) 

In [24]:
#Ejercicio 3: Estimación completa
#Probabilidad de tener una enfermeda cardiaca para toda la muestra de antecedentes familiares

In [39]:
m1_logit_all = smf.logit('chd ~ sbp+tobacco+ldl+adiposity+famhist+typea+obesity+alcohol+age', df).fit()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


In [43]:
def concise_summary_02(mod, print_fit=True):
    #guardamos los parámetros asociados a estadísticas de ajuste
    fit = pd.DataFrame({'Statistics': mod.summary2().tables[0][2][2:],
                       'Value': mod.summary2().tables[0][3][2:]})
    # guardamos los parámetros estimados por cada regresor.
    estimates = pd.DataFrame(mod.summary2().tables[1].loc[:, 'Coef.': 'Std.Err.'])
    estimates['z']=estimates['Coef.']/estimates['Std.Err.']
    # imprimir fit es opcional
    if print_fit is True:
        print("\nGoodness of Fit statistics\n", fit)
    
    print("\nPoint Estimates\n\n", estimates)

In [44]:
concise_summary_02(m1_logit_all, False)


Point Estimates

               Coef.  Std.Err.         z
Intercept -6.150721  1.308260 -4.701451
sbp        0.006504  0.005730  1.135003
tobacco    0.079376  0.026603  2.983758
ldl        0.173924  0.059662  2.915166
adiposity  0.018587  0.029289  0.634583
famhist    0.925370  0.227894  4.060530
typea      0.039595  0.012320  3.213823
obesity   -0.062910  0.044248 -1.421764
alcohol    0.000122  0.004483  0.027137
age        0.045225  0.012130  3.728464


In [47]:
#para esta muestra, tobacco, ldl, famhist, typea y age > 1.96 no son fuertemente determinantes para
#implicar en una enfermedad coronaria

In [50]:
#Ejercicio 4: Estimación de perfiles

In [56]:
#La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.
# .... disculpa no entendí muy bien a que se refiere "caracteristicas similares a la muestra"
# sacaré datos de la primera fila del csv
estimate_01 = m1_logit_all.params['Intercept']+(m1_logit_all.params['sbp'] * 160) +(m1_logit_all.params['tobacco'] * 12)+(m1_logit_all.params['ldl'] * 5.73)+(m1_logit_all.params['adiposity'] * 23.11)+(m1_logit_all.params['famhist'] * 1)+(m1_logit_all.params['typea'] * 49)+(m1_logit_all.params['obesity'] * 25)+(m1_logit_all.params['alcohol'] * 97)+(m1_logit_all.params['age'] * 50)
print(1/(1+np.exp(-estimate_01)))


0.6972860432752113


In [57]:
#La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja dens
df.describe()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,0.415584,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,0.493357,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,0.000000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,0.000000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,0.000000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,1.000000,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,1.000000,78.000000,46.580000,147.190000,64.000000,1.000000


In [60]:
estimate_02 = m1_logit_all.params['Intercept']+(m1_logit_all.params['ldl'] * 15)
print(1/(1+np.exp(-estimate_02)))

0.02814430262400695


In [61]:
#a probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.
estimate_03 = m1_logit_all.params['Intercept']+(m1_logit_all.params['ldl'] * 0.98)
print(1/(1+np.exp(-estimate_03)))

0.002521754513187771
